In [1]:
import numpy as np
import torch
import sys
import os
import time
import xlwt
import csv
import random
import math
import copy
import pickle
import gc

# masterPath = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.insert(1, os.path.join(masterPath, 'centralRepo'))
sys.path.insert(1, '../centralRepo')
from eegDataset import eegDataset
from baseModel import baseModel
from networks import *
import transforms
from saveData import fetchData
import seaborn as sns
import matplotlib.pyplot as plt
import statistics

In [2]:
config = {}
config['modelArguments'] = {'nChan': 22, 'nTime': 1000, 'dropoutP': 0.5,
                                    'nBands':9, 'm' : 32, 'temporalLayer': 'LogVarLayer',
                                    'nClass': 4, 'doWeightNorm': True}

# Training related details    
config['modelTrainArguments'] = {'stopCondi':  {'c': {'Or': {'c1': {'MaxEpoch': {'maxEpochs': 1000, 'varName' : 'epoch'}},
                                                   'c2': {'NoDecrease': {'numEpochs' : 200, 'varName': 'valInacc'}} } }},
      'classes': [0,1], 'sampler' : 'RandomSampler', 'loadBestModel': True,
      'bestVarToCheck': 'valInacc', 'continueAfterEarlystop':False,'lr': 1e-3}

config['batchSize'] = 16
nGPU = 0

In [3]:
# config['inDataPath'] = '../../data/bci42a/multiviewPython'
# config['inLabelPath'] = '../../data//bci42a/multiviewPython/dataLabels.csv'
# ROOT_PATH = '../../output/bci42a/ses2Test/fbcnet_cross_subs/'
# net = FBCNet(**config['modelArguments'])

config['inDataPath'] = '../../data/bci42a/rawPython'
config['inLabelPath'] = '../../data//bci42a/rawPython/dataLabels.csv'
ROOT_PATH = '../../output/bci42a/ses2Test/eegnet_cross_subject/'
net = eegNet(**config['modelArguments'])

In [4]:
data = eegDataset(dataPath = config['inDataPath'], dataLabelsPath= config['inLabelPath'], preloadData = False, transform= None)

In [5]:
def test_model(test_sub, data, model, session='1'):
        subIdxTest = [i for i, x in enumerate(data.labels) if x[3] in test_sub and x[4] == session]
        testData = copy.deepcopy(data)
        testData.createPartialDataset(subIdxTest, loadNonLoadedData = True)

        testResults, pred, act, pred_prob = model.test_preds(testData)

        del testData
        gc.collect()

        return testResults, pred, act, pred_prob

In [6]:
subs = list(range(9))
accs = []
all_preds = {}
all_pred_probs = {}
all_losses = {}
all_test_results = {}
all_acts = {}
all_train_preds = {}
all_train_pred_probs = {}
all_train_losses = {}
all_train_results = {}
for test_sub in subs:
    test_sub_str = '00' + str(test_sub + 1)
    preds = []
    pred_probs = []
    train_preds = []
    train_pred_probs = []
    results = {}
    train_results = {}
    train_losses = []
    print("Testing subject " + test_sub_str + " with all models...")
    for sub in subs:
        if(sub == test_sub):
            continue
#             state_dict_path = os.path.join(ROOT_PATH, 'sub' + str(sub), 'network_state_dict.pth'
        else:
            state_dict_path = os.path.join(ROOT_PATH, 'sub' + str(sub), 'finetuned_sub' + str(test_sub), 'network_state_dict.pth')
    #     print(state_dict_path)
        netInitState = torch.load(state_dict_path)
        net.load_state_dict(netInitState, strict=False)
        outPathSub = None
        model = baseModel(net=net, resultsSavePath=outPathSub, batchSize= config['batchSize'], nGPU = nGPU)
        testResults, pred, act, pred_prob = test_model(test_sub_str, data, model, session='0')
        train_preds.append(pred)
        train_pred_probs.append(pred_prob)
        train_results[sub] = testResults
        train_losses.append(testResults['loss'])
        
        testResults, pred, act, pred_prob = test_model(test_sub_str, data, model)
        preds.append(pred)
        pred_probs.append(pred_prob)
        results[sub] = testResults
        print("Results for model", sub, testResults)
    
    correct = 0
    for i in range(len(act)):
        for pred in preds:
            if(pred[i] == act[i]):
                correct += 1
                break
    
    all_preds[test_sub] = preds
    all_pred_probs[test_sub] = pred_probs
    all_acts[test_sub] = act
    all_test_results[test_sub] = results
    
    all_train_preds[test_sub] = train_preds
    all_train_pred_probs[test_sub] = train_pred_probs
    all_train_losses[test_sub] = train_losses
    all_train_results[test_sub] = train_results
    
    acc = correct/len(act)
    accs.append(acc)
    print("Theoretical max accuracy:", acc)

Testing subject 001 with all models...
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model 1 {'acc': 0.5034722222222222, 'cm': array([[24, 21, 15, 12],
       [17, 41,  7,  7],
       [14, 10, 40,  8],
       [ 5, 20,  7, 40]]), 'loss': 1.4141811794704862}
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model 2 {'acc': 0.7013888888888888, 'cm': array([[47, 15,  5,  5],
       [15, 52,  2,  3],
       [ 2,  3, 45, 22],
       [ 2,  4,  8, 58]]), 'loss': 0.9975721571180556}
Code will be running on device  cuda:0


...

../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs
../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model 6 {'acc': 0.5798611111111112, 'cm': array([[51, 10,  8,  3],
       [ 5, 44, 16,  7],
       [10, 11, 31, 20],
       [10,  9, 12, 41]]), 'loss': 1.3259551790025499}
Code will be running on device  cuda:0


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


Results for model 7 {'acc': 0.6597222222222222, 'cm': array([[51, 10,  5,  6],
       [ 4, 41, 13, 14],
       [11, 10, 36, 15],
       [ 3,  3,  4, 62]]), 'loss': 1.2813310623168945}
Theoretical max accuracy: 0.9513888888888888


../centralRepo/baseModel.py:565: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return predicted, actual, torch.tensor(loss).item()/totalCount, predicted_probs


In [7]:
accs

[0.9479166666666666,
 0.8819444444444444,
 0.9652777777777778,
 0.9097222222222222,
 0.96875,
 0.9305555555555556,
 0.9548611111111112,
 0.9444444444444444,
 0.9513888888888888]

In [8]:
avg_acc = sum(accs) / len(accs)
print("Average theoretical max accuracy:", avg_acc)

Average theoretical max accuracy: 0.939429012345679


In [9]:
accs.append(avg_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in accs])
print(tabstr)

94.792 & 88.194 & 96.528 & 90.972 & 96.875 & 93.056 & 95.486 & 94.444 & 95.139 & 93.943


In [10]:
# sub_spec_accs = [all_test_results[i][i]['acc'] for i in range(9)]
# print(sub_spec_accs)
# print("Subject specific average accuracy:", sum(sub_spec_accs) / len(sub_spec_accs))
# print("Subject specific median accuracy:", statistics.median(sub_spec_accs))

In [11]:
voted_accs = []
for test_sub in subs:
    voted_pred = []
#     test_sub = 1
    preds = all_preds[test_sub]
    act = all_acts[test_sub]
    for i in range(len(preds[0])):
        votes = {l:0 for l in range(4)}
        for pred in preds:
            votes[pred[i]]+=1
        voted_pred.append(max(votes, key=votes.get))

    correct = 0
    for pred, act_val in zip(voted_pred, act):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(act)
    voted_accs.append(voted_acc)
    print("Voted accuracy:", voted_acc)
    
avg_voted_acc = sum(voted_accs) / len(voted_accs)
print("Average voted accuracy:", avg_voted_acc)
print("Median voted accuracy:", statistics.median(voted_accs))
voted_accs.append(avg_voted_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in voted_accs])
print(tabstr)

Voted accuracy: 0.7083333333333334
Voted accuracy: 0.5659722222222222
Voted accuracy: 0.7708333333333334
Voted accuracy: 0.6423611111111112
Voted accuracy: 0.7048611111111112
Voted accuracy: 0.5798611111111112
Voted accuracy: 0.7361111111111112
Voted accuracy: 0.7083333333333334
Voted accuracy: 0.6736111111111112
Average voted accuracy: 0.6766975308641976
Median voted accuracy: 0.7048611111111112
70.833 & 56.597 & 77.083 & 64.236 & 70.486 & 57.986 & 73.611 & 70.833 & 67.361 & 67.670


### Soft probability combine

In [12]:
comb_accs = []
for test_sub in subs:
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Soft probability combined accuracy:", voted_acc)

avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Soft probability combined accuracy: 0.7743055555555556
Soft probability combined accuracy: 0.5694444444444444
Soft probability combined accuracy: 0.8125
Soft probability combined accuracy: 0.65625
Soft probability combined accuracy: 0.7395833333333334
Soft probability combined accuracy: 0.6180555555555556
Soft probability combined accuracy: 0.7430555555555556
Soft probability combined accuracy: 0.7256944444444444
Soft probability combined accuracy: 0.6701388888888888
Average combined accuracy: 0.7010030864197532
Median combined accuracy: 0.7256944444444444
77.431 & 56.944 & 81.250 & 65.625 & 73.958 & 61.806 & 74.306 & 72.569 & 67.014 & 70.100


In [13]:
cms = {test_sub: {train_sub: results['cm'] for train_sub, results in test_results.items()} for test_sub, test_results in all_train_results.items()}

In [14]:
metrics = {test_sub: {train_sub: np.array([cm[i][i]/np.sum(cm[i,:]) for i in range(4)]) for train_sub, cm in test_results.items()} for test_sub, test_results in cms.items()}

In [15]:
recall_sums = {test_sub: np.array([np.sum(recalls) for train_sub, recalls in test_results.items()]) for test_sub, test_results in metrics.items()}

In [16]:
recall_sums[0]

array([3.94444444, 4.        , 3.94444444, 3.86111111, 3.94444444,
       3.97222222, 3.97222222, 3.93055556])

In [17]:
np.argsort(recall_sums[0])

array([3, 7, 0, 2, 4, 5, 6, 1])

In [18]:
comb_accs = []
for test_sub in subs:
    mlist = list(range(9))
    mlist.pop(test_sub)
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
#     print(pred_probs)
    for i in range(len(pred_probs)):
#         print(pred_probs[i])
        for j in range(len(pred_probs[i])):
            pred_probs[i][j] = np.multiply(pred_probs[i][j], metrics[test_sub][mlist[i]])
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Soft probability combined accuracy:", voted_acc)

avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Soft probability combined accuracy: 0.7777777777777778
Soft probability combined accuracy: 0.5729166666666666
Soft probability combined accuracy: 0.8125
Soft probability combined accuracy: 0.65625
Soft probability combined accuracy: 0.7361111111111112
Soft probability combined accuracy: 0.6180555555555556
Soft probability combined accuracy: 0.7430555555555556
Soft probability combined accuracy: 0.7256944444444444
Soft probability combined accuracy: 0.6701388888888888
Average combined accuracy: 0.7013888888888888
Median combined accuracy: 0.7256944444444444
77.778 & 57.292 & 81.250 & 65.625 & 73.611 & 61.806 & 74.306 & 72.569 & 67.014 & 70.139


In [19]:
voted_accs = []
for test_sub in subs:
    voted_pred = []
    mlist = list(range(9))
    mlist.pop(test_sub)
#     test_sub = 1
    preds = all_preds[test_sub]
    act = all_acts[test_sub]
    for i in range(len(preds[0])):
        votes = {l:0 for l in range(4)}
        for j, pred in enumerate(preds):
            votes[pred[i]]+=metrics[test_sub][mlist[j]][pred[i]]
        voted_pred.append(max(votes, key=votes.get))

    correct = 0
    for pred, act_val in zip(voted_pred, act):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(act)
    voted_accs.append(voted_acc)
    print("Voted accuracy:", voted_acc)
    
avg_voted_acc = sum(voted_accs) / len(voted_accs)
print("Average voted accuracy:", avg_voted_acc)
print("Median voted accuracy:", statistics.median(voted_accs))
voted_accs.append(avg_voted_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in voted_accs])
print(tabstr)

Voted accuracy: 0.7048611111111112
Voted accuracy: 0.5729166666666666
Voted accuracy: 0.7708333333333334
Voted accuracy: 0.6458333333333334
Voted accuracy: 0.7048611111111112
Voted accuracy: 0.59375
Voted accuracy: 0.7291666666666666
Voted accuracy: 0.7048611111111112
Voted accuracy: 0.6805555555555556
Average voted accuracy: 0.6786265432098766
Median voted accuracy: 0.7048611111111112
70.486 & 57.292 & 77.083 & 64.583 & 70.486 & 59.375 & 72.917 & 70.486 & 68.056 & 67.863


In [20]:
comb_accs = []
for test_sub in subs:
    mlist = list(range(9))
    mlist.pop(test_sub)
    sorted_recall_idx = np.argsort(recall_sums[test_sub])
    # print(sorted_recall_idx)
    # print(sorted_recall_idx[-5:])
    pred_probs = np.array(all_pred_probs[test_sub])
    n, h, w, c = pred_probs.shape
    pred_probs = np.reshape(pred_probs, (n, h*w, c))
#     print(pred_probs.shape)
    pred_probs = np.take(pred_probs, sorted_recall_idx[-3:], 0)
#     print(pred_probs.shape)
    for i in range(len(pred_probs)):
        for j in range(len(pred_probs[i])):
            pred_probs[i][j] = np.multiply(pred_probs[i][j], metrics[test_sub][mlist[i]])
    class_probs = np.sum(pred_probs, axis=0)
    soft_preds = np.argmax(class_probs, axis=1)
    correct = 0
    for pred, act_val in zip(soft_preds, all_acts[test_sub]):
        if(pred == act_val):
            correct += 1
    voted_acc = correct / len(all_acts[test_sub])
    comb_accs.append(voted_acc)
    print("Top recall scaled soft probability combined accuracy:", voted_acc)
    
avg_comb_acc = sum(comb_accs) / len(comb_accs)
print("Average combined accuracy:", avg_comb_acc)
print("Median combined accuracy:", statistics.median(comb_accs))
comb_accs.append(avg_comb_acc)
tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
print(tabstr)

Top recall scaled soft probability combined accuracy: 0.7256944444444444
Top recall scaled soft probability combined accuracy: 0.5833333333333334
Top recall scaled soft probability combined accuracy: 0.8090277777777778
Top recall scaled soft probability combined accuracy: 0.6145833333333334
Top recall scaled soft probability combined accuracy: 0.7326388888888888
Top recall scaled soft probability combined accuracy: 0.5763888888888888
Top recall scaled soft probability combined accuracy: 0.7361111111111112
Top recall scaled soft probability combined accuracy: 0.7083333333333334
Top recall scaled soft probability combined accuracy: 0.6458333333333334
Average combined accuracy: 0.681327160493827
Median combined accuracy: 0.7083333333333334
72.569 & 58.333 & 80.903 & 61.458 & 73.264 & 57.639 & 73.611 & 70.833 & 64.583 & 68.133


In [21]:
for i in range(2,8):
    comb_accs = []
    print("Top-" + str(i))
    for test_sub in subs:
        mlist = list(range(9))
        mlist.pop(test_sub)
        sorted_recall_idx = np.argsort(recall_sums[test_sub])
        # print(sorted_recall_idx)
        # print(sorted_recall_idx[-5:])
        pred_probs = np.array(all_pred_probs[test_sub])
        n, h, w, c = pred_probs.shape
        pred_probs = np.reshape(pred_probs, (n, h*w, c))
    #     print(pred_probs.shape)
        pred_probs = np.take(pred_probs, sorted_recall_idx[-i:], 0)
    #     print(pred_probs.shape)
        for i in range(len(pred_probs)):
            for j in range(len(pred_probs[i])):
                pred_probs[i][j] = np.multiply(pred_probs[i][j], metrics[test_sub][mlist[i]])
        class_probs = np.sum(pred_probs, axis=0)
        soft_preds = np.argmax(class_probs, axis=1)
        correct = 0
        for pred, act_val in zip(soft_preds, all_acts[test_sub]):
            if(pred == act_val):
                correct += 1
        voted_acc = correct / len(all_acts[test_sub])
        comb_accs.append(voted_acc)
        print("Top recall scaled soft probability combined accuracy:", voted_acc)

    avg_comb_acc = sum(comb_accs) / len(comb_accs)
    print("Average combined accuracy:", avg_comb_acc)
    print("Median combined accuracy:", statistics.median(comb_accs))
    comb_accs.append(avg_comb_acc)
    tabstr = ' & '.join(["{0:.3f}".format(x*100) for x in comb_accs])
    print(tabstr)
    print()
    print()

Top-2
Top recall scaled soft probability combined accuracy: 0.7465277777777778
Top recall scaled soft probability combined accuracy: 0.5138888888888888
Top recall scaled soft probability combined accuracy: 0.8125
Top recall scaled soft probability combined accuracy: 0.65625
Top recall scaled soft probability combined accuracy: 0.7152777777777778
Top recall scaled soft probability combined accuracy: 0.5902777777777778
Top recall scaled soft probability combined accuracy: 0.7638888888888888
Top recall scaled soft probability combined accuracy: 0.7083333333333334
Top recall scaled soft probability combined accuracy: 0.6215277777777778
Average combined accuracy: 0.6809413580246912
Median combined accuracy: 0.7083333333333334
74.653 & 51.389 & 81.250 & 65.625 & 71.528 & 59.028 & 76.389 & 70.833 & 62.153 & 68.094


Top-3
Top recall scaled soft probability combined accuracy: 0.7256944444444444
Top recall scaled soft probability combined accuracy: 0.5868055555555556
Top recall scaled soft prob